In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from relife import Weibull
pd.options.plotting.backend = "plotly"

In [69]:
duree_construction = 3
duree_projet = 30
duree_amortissement = 10
CAPEX = 1200000
OPEX = CAPEX * 3/100
dette = CAPEX / 3
duree_dette = duree_construction + duree_amortissement
taux_actualisation = 5
taux_interets = 2
prod_annuelle_MWh = 2200
Cfd_price = 75
taux_imposition = 30/100

In [78]:
df = pd.DataFrame(columns=[k for k in range(-duree_construction+1,duree_projet+1)])
df.loc["Coef. actualisation"] = [(1+taux_actualisation/100)**k for k in df.columns]
df.loc["CAPEX",:0] = -CAPEX/duree_construction
df.loc["CAPEX",20:21] = -CAPEX/12
df.loc["OPEX",1:] = -OPEX
# df.loc["TOTEX"] = df.loc["CAPEX"] + df.loc["OPEX"]
df.loc["Prod. (MWh)",1:] = prod_annuelle_MWh
df.loc["Revenus"] = Cfd_price * df.loc["Prod. (MWh)"]
df.loc["EBITDA"] = df.loc[["OPEX","Revenus"]].sum()
df.loc["Amortissements",1:duree_amortissement] = - CAPEX/duree_amortissement
df.loc["EBIT"] = df.loc[["EBITDA","Amortissements"]].sum()
df.loc["Dette BOP",:duree_dette-duree_construction+1] = np.linspace(dette,0,duree_dette+1)
df.loc["Tirage",-2] = dette
df.loc["Remboursement",:duree_dette-duree_construction] = - dette/duree_dette
df.loc["Dette EOP"] = df.loc[["Dette BOP", "Remboursement"]].sum()
# df.loc["Dette EOP",:duree_dette-duree_construction+1] = df.loc["Dette BOP",:duree_dette-duree_construction]
df.loc["Intérêts financiers"] = - df.loc["Dette BOP",:duree_dette-duree_construction+1] * 2 / 100
df.loc["Résultat avant IS"] = df.loc[["EBIT","Intérêts financiers"]].sum()
df.loc["Impôts (IS)"] = - df.loc["Résultat avant IS"].apply(lambda x : 0 if x < 0 else x) * taux_imposition
df.loc["Résultat net"] = df.loc[["Résultat avant IS","Impôts (IS)"]].sum()
df.loc["Cash Flow"] = df.loc[["OPEX","CAPEX","Revenus","Intérêts financiers", "Impôts (IS)","Tirage"]].sum()
fig = df.transpose().plot(kind = "bar", y = ["CAPEX","OPEX","Revenus"]).update_layout(hovermode='x')
fig.add_trace(
    go.Scatter(
        x = df.columns,
        y = df.loc["Cash Flow"],
        name = "Cash Flow",
        marker_color = "black"
    )
)
fig.add_trace(
    go.Scatter(
        x = df.columns,
        y = df.loc["Dette BOP"],
        name = "Endettement",
        marker_color = "orange"
    )
)

In [76]:
df.transpose().plot(kind = "bar", y = ["Dette EOP"]).update_layout(hovermode='x')

In [36]:
dd = df / df.loc["Coef. actualisation"]
dd

,-2,-1,0,1,2,3,4,5,6,7,...,21,22,23,24,25,26,27,28,29,30
Coef. actualisation,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
CAPEX,-441000.0,-420000.0,-400000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-35894.236464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OPEX,NaN,NaN,NaN,-34285.714286,-32653.061224,-31098.153547,-29617.289093,-28206.941993,-26863.754279,-25584.527885,...,-12921.925127,-12306.595359,-11720.567009,-11162.444770,-10630.899781,-10124.666458,-9642.539484,-9183.370937,-8746.067559,-8329.588152
Prod. (MWh),NaN,NaN,NaN,2095.238095,1995.464853,1900.442717,1809.945445,1723.757566,1641.673873,1563.498926,...,789.673202,752.069716,716.256873,682.149403,649.666098,618.729617,589.266302,561.206002,534.481906,509.030387
Revenus,NaN,NaN,NaN,157142.857143,149659.863946,142533.203758,135745.908341,129281.817467,123125.540445,117262.419471,...,59225.490166,56405.228729,53719.265456,51161.205197,48724.957330,46404.721267,44194.972635,42090.450129,40086.142980,38177.279028
EBITDA,0.0,0.0,0.0,122857.142857,117006.802721,111435.050211,106128.619248,101074.875474,96261.786166,91677.891587,...,46303.565039,44098.633370,41998.698448,39998.760426,38094.057549,36280.054809,34552.433151,32907.079191,31340.075420,29847.690877
Amortissements,NaN,NaN,NaN,-114285.714286,-108843.537415,-103660.511824,-98724.296975,-94023.139976,-89545.847596,-85281.759616,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EBIT,0.0,0.0,0.0,8571.428571,8163.265306,7774.538387,7404.322273,7051.735498,6715.938570,6396.131971,...,46303.565039,44098.633370,41998.698448,39998.760426,38094.057549,36280.054809,34552.433151,32907.079191,31340.075420,29847.690877
Intérêts financiers,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Résultat avant IS,0.0,0.0,0.0,8571.428571,8163.265306,7774.538387,7404.322273,7051.735498,6715.938570,6396.131971,...,46303.565039,44098.633370,41998.698448,39998.760426,38094.057549,36280.054809,34552.433151,32907.079191,31340.075420,29847.690877


In [37]:
fig = dd.transpose().plot(kind = "bar", y = ["CAPEX","OPEX","Revenus","Impôts (IS)"]).update_layout(hovermode='x')
fig.add_trace(
    go.Scatter(
        x = dd.columns,
        y = dd.loc["Cash Flow"],
        name = "Cash Flow",
        marker_color = "black"
    )
)

In [38]:
def groupement_année(k):
    if k<=0:
        return "Années négatives"
    else:
        return f"Années {((k-1)//5)*5+1}-{((k-1)//5+1)*5}"
de = pd.melt(dd.loc[["OPEX","CAPEX","Revenus","Impôts (IS)"]].reset_index(),id_vars="index").rename({"variable":"Année projet","index":"Type de flux"},axis=1)
de["value"] = de["value"].abs().round()
de["Période projet"] = de["Année projet"].apply(groupement_année)
de

,Type de flux,Année projet,value,Période projet
0,OPEX,-2,NaN,Années négatives
1,CAPEX,-2,441000.0,Années négatives
2,Revenus,-2,NaN,Années négatives
3,Impôts (IS),-2,0.0,Années négatives
4,OPEX,-1,NaN,Années négatives
...,...,...,...,...
127,Impôts (IS),29,9402.0,Années 26-30
128,OPEX,30,8330.0,Années 26-30
129,CAPEX,30,NaN,Années 26-30
130,Revenus,30,38177.0,Années 26-30


In [39]:
fig = px.treemap(
    de, 
    path = [px.Constant("Ensemble du projet"), "Type de flux", "Période projet", "Année projet"], 
    values="value",
    maxdepth=3
)
fig.data[0].textinfo = 'label+value'
fig.update_traces(root_color="lightgrey")
fig.update_layout(title = "Valeur actualisé des flux du projet")
fig.show()

In [40]:
aa = df.loc["CAPEX"]/df.loc["Coef. actualisation"]
aa.sum()

-1334583.1847513954

In [41]:
aa = df.loc["Cash Flow"]/df.loc["Coef. actualisation"]
aa.sum()

331531.60042677744

In [43]:
import numpy_financial as npf

In [48]:
npf.npv(5/100,df.loc["Cash Flow"])*(1+5/100)**2

331531.60042677727

In [45]:
npf.irr(df.loc["Cash Flow"])*100

7.30015972674376